In [3]:
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets, transforms
import torchmetrics
import pytorch_lightning as pl

In [4]:
# hyperparams
in_channels = 3
num_classes = 10
learning_rate = 0.001
batch_size = 32
num_epochs = 10

In [5]:
image_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224))
])

In [6]:
train_dataset = datasets.ImageFolder("/Users/jose.laruta/dev/topics/dataset/cats/train", transform=image_transforms)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=8, persistent_workers=True)

test_dataset = datasets.ImageFolder("/Users/jose.laruta/dev/topics/dataset/cats/test", transform=image_transforms)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, num_workers=8, persistent_workers=True)


In [15]:
class CNN(pl.LightningModule):
    def __init__(self, in_channels=3, num_classes=10, lr=0.001):
        super().__init__()
        self.lr = lr
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.pool = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv4 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.fc1 = nn.Linear(64 * 14 * 14, num_classes)

        self.features = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            self.pool,
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            self.pool,
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            self.pool,
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            self.pool,
        )
        self.head = nn.Sequential(
            self.fc1,
        )

        self.loss_fn = nn.CrossEntropyLoss()
        self.accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes)
        self.f1_score = torchmetrics.F1Score(task="multiclass", num_classes=num_classes)

    def forward(self, x):
        x = self.features(x)
        # flatten
        x = x.reshape(x.shape[0], -1)
        x = self.head(x)
        return x
    
    def _common_step(self, batch, batch_idx):
        x, y = batch
        scores = self.forward(x)
        loss = self.loss_fn(scores, y)
        return loss, scores, y
    
    def training_step(self, batch, batch_idx):
        loss, scores, y = self._common_step(batch, batch_idx)
        accuracy = self.accuracy(scores, y)
        f1_score = self.f1_score(scores, y)

        self.log_dict(
            {"train_loss": loss, "train_accuracy": accuracy, "train_f1_score": f1_score},
            on_step=False,
            on_epoch=True,
            prog_bar=True
        )
        return loss

    def validation_step(self, batch, batch_idx):
        loss, scores, y = self._common_step(batch, batch_idx)
        self.log("val_loss", loss)
        return loss

    def test_step(self, batch, batch_idx):
        loss, scores, y = self._common_step(batch, batch_idx)
        accuracy = self.accuracy(scores, y)
        f1_score = self.f1_score(scores, y)

        self.log_dict(
            {"test_loss": loss, "test_accuracy": accuracy, "test_f1_score": f1_score},
            on_step=False,
            on_epoch=True,
            prog_bar=True
        )
        return loss
    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.lr)

In [6]:
class MyResnet18(pl.LightningModule):
    def __init__(self, num_classes=5, freeze_backbone=False):
        super().__init__()
        self.weights = torchvision.models.ResNet18_Weights.DEFAULT
        self.backbone = torchvision.models.resnet18(weights=self.weights)
        self.preprocess = self.weights.transforms()
        if freeze_backbone:
            for param in self.parameters():
                param.requires_grad = False
        self.fc = nn.Linear(512, num_classes)
        self.backbone.fc = self.fc
        self.loss_fn = nn.CrossEntropyLoss()
        self.accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes)
        self.f1_score = torchmetrics.F1Score(task="multiclass", num_classes=num_classes)

    def forward(self, x):
        return self.backbone(x)

    def training_step(self, batch, batch_idx):
        loss, scores, y = self._common_step(batch, batch_idx)
        accuracy = self.accuracy(scores, y)
        f1_score = self.f1_score(scores, y)

        self.log_dict(
            {"train_loss": loss, "train_accuracy": accuracy, "train_f1_score": f1_score},
            on_step=False,
            on_epoch=True,
            prog_bar=True
        )
        return loss

    def validation_step(self, batch, batch_idx):
        loss, scores, y = self._common_step(batch, batch_idx)
        self.log("val_loss", loss)
        return loss

    def test_step(self, batch, batch_idx):
        loss, scores, y = self._common_step(batch, batch_idx)
        accuracy = self.accuracy(scores, y)
        f1_score = self.f1_score(scores, y)

        self.log_dict(
            {"test_loss": loss, "test_accuracy": accuracy, "test_f1_score": f1_score},
            on_step=False,
            on_epoch=True,
            prog_bar=True
        )
        return loss

    def _common_step(self, batch, batch_idx):
        x, y = batch
        scores = self.forward(x)
        loss = self.loss_fn(scores, y)
        return loss, scores, y

    def predict_step(self, batch, batch_idx):
        x, y = batch
        scores = self.forward(x)
        preds = torch.argmax(scores, dim=1)
        return preds

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=0.001)

In [16]:
model = CNN(num_classes=num_classes, lr=learning_rate)
# formato:      batch_size, n_channels, x, y
x = torch.randn(32, 3, 224, 224)
print(x.shape)
print(model(x).shape)

torch.Size([32, 3, 224, 224])
torch.Size([32, 10])


In [17]:
torch.set_float32_matmul_precision('high')
trainer = pl.Trainer(
    # accelerator="gpu", # para GPUs Nvidia: "gpu"
    # devices=1,          
    min_epochs=1,
    max_epochs=num_epochs,
    # precision="bf16-mixed"
)
trainer.fit(model, train_loader, test_loader)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

   | Name     | Type               | Params | Mode 
---------------------------------------------------------
0  | conv1    | Conv2d             | 224    | train
1  | pool     | MaxPool2d          | 0      | train
2  | conv2    | Conv2d             | 1.2 K  | train
3  | conv3    | Conv2d             | 4.6 K  | train
4  | conv4    | Conv2d             | 18.5 K | train
5  | fc1      | Linear             | 125 K  | train
6  | features | Sequential         | 24.5 K | train
7  | head     | Sequential         | 125 K  | train
8  | loss_fn  | CrossEntropyLoss   | 0      | train
9  | accuracy | MulticlassAccuracy | 0      | train
10 | f1_score | MulticlassF1Score  | 0      | tra

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [18]:
trainer.test(model, test_loader)

Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy         0.6200000047683716
      test_f1_score         0.6200000047683716
        test_loss           1.2443379163742065
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 1.2443379163742065,
  'test_accuracy': 0.6200000047683716,
  'test_f1_score': 0.6200000047683716}]

In [17]:
from torchvision.io import read_image
model.eval()
img = read_image("tiger1.jpg")
batch = model.preprocess(img).unsqueeze(0)
prediction = model(batch).squeeze(0).softmax(0)
class_id = prediction.argmax().item()
score = prediction[class_id].item()
category_name = train_dataset.classes[class_id]
print(prediction, class_id, score, category_name)

tensor([0.0049, 0.0019, 0.0572, 0.0037, 0.0697, 0.0190, 0.0073, 0.0029, 0.0085,
        0.8249], grad_fn=<SoftmaxBackward0>) 9 0.8249470591545105 TIGER
